# 📈 Google (GOOGL) Stock Price Prediction with Advanced ML Models

## 🎯 Objective
Predict Google stock prices using **7 different machine learning models** and provide **clear BUY/SELL recommendations** for paper trading.

### Features:
- ✅ **10 Years of Historical Data** for maximum accuracy
- ✅ **7+ ML Models** (Linear, Ridge, Lasso, Random Forest, XGBoost, Gradient Boosting, Extra Trees, SVR)
- ✅ **40+ Technical Indicators** (SMA, EMA, RSI, MACD, Bollinger Bands, etc.)
- ✅ **Ensemble Predictions** (weighted average of all models)
- ✅ **Buy/Sell Signal Generation** with confidence scores
- ✅ **Paper Trading Ready** recommendations
- ✅ **Comprehensive Visualizations**

⚠️ **DISCLAIMER**: This is for educational and paper trading purposes only. Not financial advice!

## 1. Import Libraries

In [ ]:
# System imports
import sys
sys.path.insert(0, '../src')

# Data processing
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# ML models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

# Project modules
from data_collector import StockDataCollector
from feature_engineering import FeatureEngineer
from visualizer import StockVisualizer

print("✅ All libraries imported successfully!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Configuration

In [ ]:
# Stock configuration
TICKER = 'GOOGL'  # Google stock
PERIOD = '10y'    # 10 years of data for maximum accuracy
TEST_SIZE = 0.2   # 20% for testing

# Model configuration
RANDOM_STATE = 42
N_ESTIMATORS = 200  # More trees for better accuracy

# Trading signal configuration
BUY_THRESHOLD = 0.02   # 2% predicted increase = BUY signal
SELL_THRESHOLD = -0.01 # 1% predicted decrease = SELL signal
CONFIDENCE_THRESHOLD = 0.7  # 70% model agreement required

print(f"🎯 Target Stock: {TICKER}")
print(f"📊 Data Period: {PERIOD}")
print(f"🤖 Number of Estimators: {N_ESTIMATORS}")
print(f"📈 Buy Threshold: {BUY_THRESHOLD*100}%")
print(f"📉 Sell Threshold: {SELL_THRESHOLD*100}%")

## 3. Data Collection

Fetching 10 years of Google stock data for maximum prediction accuracy.

In [ ]:
# Initialize data collector
collector = StockDataCollector(data_dir='../data')

# Fetch Google stock data
print(f"📥 Fetching {TICKER} data for {PERIOD}...")
df = collector.fetch_stock_data(TICKER, period=PERIOD)

print(f"\n✅ Data loaded successfully!")
print(f"📊 Shape: {df.shape}")
print(f"📅 Date Range: {df.index[0]} to {df.index[-1]}")
print(f"💰 Latest Close Price: ${df['Close'].iloc[-1]:.2f}")

df.head()

In [ ]:
# Get stock information
info = collector.get_stock_info(TICKER)

print("\n" + "="*60)
print(f"📊 {TICKER} - COMPANY INFORMATION")
print("="*60)
for key, value in info.items():
    print(f"  {key.replace('_', ' ').title()}: {value}")
print("="*60)

## 4. Exploratory Data Analysis

In [ ]:
# Statistical summary
print("📊 Statistical Summary:\n")
print(df.describe())

# Price statistics
print(f"\n💰 Price Statistics:")
print(f"  All-Time High: ${df['High'].max():.2f}")
print(f"  All-Time Low: ${df['Low'].min():.2f}")
print(f"  Average Close: ${df['Close'].mean():.2f}")
print(f"  Volatility (Std): ${df['Close'].std():.2f}")
print(f"  Total Return: {((df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1) * 100:.2f}%")

In [ ]:
# Initialize visualizer
viz = StockVisualizer()

# Plot stock price history
viz.plot_stock_price(df, TICKER)

In [ ]:
# Volume and price analysis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))

# Price chart
ax1.plot(df.index, df['Close'], label='Close Price', color='blue', linewidth=2)
ax1.fill_between(df.index, df['Low'], df['High'], alpha=0.3, color='lightblue')
ax1.set_title(f'{TICKER} - Price History ({PERIOD})', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Volume chart
ax2.bar(df.index, df['Volume'], color='green', alpha=0.6)
ax2.set_title('Trading Volume', fontsize=14, fontweight='bold')
ax2.set_ylabel('Volume', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Feature Engineering

Creating 40+ technical indicators for better prediction accuracy.

In [ ]:
# Create features
print("🔧 Engineering features...\n")
engineer = FeatureEngineer(df)
engineer.add_all_features()
features_df = engineer.get_feature_dataframe()
feature_names = engineer.get_feature_names()

print(f"✅ Feature engineering complete!")
print(f"📊 Features created: {len(feature_names)}")
print(f"📐 Dataset shape: {features_df.shape}")
print(f"\n📝 Feature List:")
for i, feat in enumerate(feature_names, 1):
    print(f"  {i}. {feat}")

In [ ]:
# Check for missing values
print("🔍 Checking data quality...\n")
missing_before = features_df.isnull().sum().sum()
features_df = features_df.dropna()
missing_after = features_df.isnull().sum().sum()

print(f"  Missing values before: {missing_before}")
print(f"  Missing values after: {missing_after}")
print(f"  Final dataset shape: {features_df.shape}")
print(f"\n✅ Data is clean and ready for modeling!")

In [ ]:
# Visualize key technical indicators
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Price with Moving Averages
axes[0].plot(features_df.index, features_df['Close'], label='Close Price', linewidth=2)
axes[0].plot(features_df.index, features_df['SMA_20'], label='SMA 20', linestyle='--', alpha=0.7)
axes[0].plot(features_df.index, features_df['SMA_50'], label='SMA 50', linestyle='--', alpha=0.7)
axes[0].set_title('Price with Moving Averages', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(features_df.index, features_df['RSI'], label='RSI', color='purple', linewidth=2)
axes[1].axhline(y=70, color='r', linestyle='--', label='Overbought (70)')
axes[1].axhline(y=30, color='g', linestyle='--', label='Oversold (30)')
axes[1].set_title('Relative Strength Index (RSI)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('RSI')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MACD
axes[2].plot(features_df.index, features_df['MACD'], label='MACD', linewidth=2)
axes[2].plot(features_df.index, features_df['MACD_Signal'], label='Signal', linewidth=2)
axes[2].bar(features_df.index, features_df['MACD_Histogram'], label='Histogram', alpha=0.3)
axes[2].set_title('MACD Indicator', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Prepare Training and Test Data

In [ ]:
# Prepare features and target
X = features_df[feature_names].values
y = features_df['Target'].values

# Time series split (no shuffling)
split_idx = int(len(X) * (1 - TEST_SIZE))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Get dates for test set
test_dates = features_df.index[split_idx:]

print("📊 Data Split Summary:")
print(f"  Total samples: {len(X)}")
print(f"  Training samples: {len(X_train)} ({(1-TEST_SIZE)*100:.0f}%)")
print(f"  Test samples: {len(X_test)} ({TEST_SIZE*100:.0f}%)")
print(f"  Features: {len(feature_names)}")
print(f"  Train date range: {features_df.index[0]} to {features_df.index[split_idx-1]}")
print(f"  Test date range: {test_dates[0]} to {test_dates[-1]}")
print(f"\n✅ Data prepared for training!")

## 7. Train Multiple ML Models

Training 8 different models to maximize prediction accuracy and provide ensemble predictions.

In [ ]:
# Dictionary to store models and results
models = {}
predictions = {}
metrics = {}

# Define all models
model_configs = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=RANDOM_STATE),
    'Lasso Regression': Lasso(alpha=0.1, random_state=RANDOM_STATE),
    'Random Forest': RandomForestRegressor(
        n_estimators=N_ESTIMATORS, 
        max_depth=20, 
        random_state=RANDOM_STATE, 
        n_jobs=-1
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=N_ESTIMATORS, 
        learning_rate=0.1, 
        max_depth=7, 
        random_state=RANDOM_STATE,
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=N_ESTIMATORS, 
        learning_rate=0.1, 
        max_depth=5, 
        random_state=RANDOM_STATE
    ),
    'Extra Trees': ExtraTreesRegressor(
        n_estimators=N_ESTIMATORS, 
        max_depth=20, 
        random_state=RANDOM_STATE, 
        n_jobs=-1
    ),
    'AdaBoost': AdaBoostRegressor(
        n_estimators=100, 
        learning_rate=0.1, 
        random_state=RANDOM_STATE
    )
}

print("🤖 Training 8 ML Models...\n")
print("="*60)

for name, model in model_configs.items():
    print(f"\n🔄 Training {name}...")
    
    # Train model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
    # Store results
    models[name] = model
    predictions[name] = y_pred
    metrics[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape
    }
    
    print(f"  ✅ {name} trained!")
    print(f"     R² Score: {r2:.4f}")
    print(f"     RMSE: ${rmse:.2f}")
    print(f"     MAE: ${mae:.2f}")
    print(f"     MAPE: {mape:.2f}%")

print("\n" + "="*60)
print("✅ All models trained successfully!")
print("="*60)

## 8. Compare Model Performance

In [ ]:
# Create metrics comparison dataframe
metrics_df = pd.DataFrame(metrics).T
metrics_df = metrics_df.sort_values('R2', ascending=False)

print("\n📊 MODEL PERFORMANCE COMPARISON")
print("="*80)
print(metrics_df.to_string())
print("="*80)

# Find best model
best_model_name = metrics_df.index[0]
best_r2 = metrics_df.loc[best_model_name, 'R2']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   R² Score: {best_r2:.4f}")
print(f"   RMSE: ${metrics_df.loc[best_model_name, 'RMSE']:.2f}")
print(f"   MAE: ${metrics_df.loc[best_model_name, 'MAE']:.2f}")
print(f"   MAPE: {metrics_df.loc[best_model_name, 'MAPE']:.2f}%")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# R² Score
metrics_df['R2'].plot(kind='barh', ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('R² Score (Higher is Better)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('R² Score')
axes[0, 0].grid(True, alpha=0.3)

# RMSE
metrics_df['RMSE'].plot(kind='barh', ax=axes[0, 1], color='salmon')
axes[0, 1].set_title('RMSE (Lower is Better)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('RMSE ($)')
axes[0, 1].grid(True, alpha=0.3)

# MAE
metrics_df['MAE'].plot(kind='barh', ax=axes[1, 0], color='lightgreen')
axes[1, 0].set_title('MAE (Lower is Better)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('MAE ($)')
axes[1, 0].grid(True, alpha=0.3)

# MAPE
metrics_df['MAPE'].plot(kind='barh', ax=axes[1, 1], color='plum')
axes[1, 1].set_title('MAPE (Lower is Better)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('MAPE (%)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot predictions vs actual for all models
fig, axes = plt.subplots(4, 2, figsize=(16, 20))
axes = axes.flatten()

for idx, (name, y_pred) in enumerate(predictions.items()):
    ax = axes[idx]
    
    ax.plot(test_dates, y_test, label='Actual', linewidth=2, color='blue', alpha=0.7)
    ax.plot(test_dates, y_pred, label='Predicted', linewidth=2, color='red', alpha=0.7, linestyle='--')
    
    r2 = metrics[name]['R2']
    ax.set_title(f'{name} (R²={r2:.4f})', fontsize=12, fontweight='bold')
    ax.set_ylabel('Price ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 9. Ensemble Prediction

Combining all models using weighted average based on R² scores for more robust predictions.

In [ ]:
# Calculate weights based on R² scores (higher R² = more weight)
r2_scores = np.array([metrics[name]['R2'] for name in predictions.keys()])
r2_scores = np.maximum(r2_scores, 0)  # Ensure non-negative
weights = r2_scores / r2_scores.sum()  # Normalize to sum to 1

# Calculate weighted ensemble prediction
ensemble_pred = np.zeros(len(y_test))
for name, y_pred in predictions.items():
    weight = weights[list(predictions.keys()).index(name)]
    ensemble_pred += weight * y_pred

# Calculate ensemble metrics
ensemble_rmse = np.sqrt(mean_squared_error(y_test, ensemble_pred))
ensemble_mae = mean_absolute_error(y_test, ensemble_pred)
ensemble_r2 = r2_score(y_test, ensemble_pred)
ensemble_mape = np.mean(np.abs((y_test - ensemble_pred) / y_test)) * 100

print("🎯 ENSEMBLE MODEL PERFORMANCE")
print("="*60)
print("\nModel Weights (based on R² scores):")
for name, weight in zip(predictions.keys(), weights):
    print(f"  {name}: {weight:.4f} ({weight*100:.2f}%)")
    
print("\nEnsemble Metrics:")
print(f"  R² Score: {ensemble_r2:.4f}")
print(f"  RMSE: ${ensemble_rmse:.2f}")
print(f"  MAE: ${ensemble_mae:.2f}")
print(f"  MAPE: {ensemble_mape:.2f}%")
print("="*60)

In [ ]:
# Plot ensemble prediction
plt.figure(figsize=(15, 6))

plt.plot(test_dates, y_test, label='Actual Price', linewidth=2.5, color='blue')
plt.plot(test_dates, ensemble_pred, label='Ensemble Prediction', linewidth=2.5, color='red', linestyle='--')

# Add confidence band
std_pred = np.std([predictions[name] for name in predictions.keys()], axis=0)
plt.fill_between(test_dates, 
                 ensemble_pred - std_pred, 
                 ensemble_pred + std_pred, 
                 alpha=0.2, color='red', label='Prediction Uncertainty')

plt.title(f'{TICKER} - Ensemble Model Prediction (R²={ensemble_r2:.4f})', 
          fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 10. Feature Importance Analysis

In [ ]:
# Get feature importance from tree-based models
tree_models = ['Random Forest', 'XGBoost', 'Gradient Boosting', 'Extra Trees']
importance_data = []

for model_name in tree_models:
    if model_name in models:
        importances = models[model_name].feature_importances_
        importance_data.append(importances)

# Average importance across all tree models
avg_importance = np.mean(importance_data, axis=0)
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': avg_importance
}).sort_values('Importance', ascending=False)

# Display top 20 features
print("\n🔍 TOP 20 MOST IMPORTANT FEATURES")
print("="*60)
print(feature_importance_df.head(20).to_string(index=False))
print("="*60)

In [ ]:
# Plot top 15 features
plt.figure(figsize=(12, 8))
top_features = feature_importance_df.head(15)
plt.barh(range(len(top_features)), top_features['Importance'], color='teal')
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Average Importance', fontsize=12)
plt.title('Top 15 Most Important Features', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 11. Next-Day Price Prediction

In [ ]:
# Get the latest data point for prediction
latest_features = features_df[feature_names].iloc[-1:].values
latest_features_scaled = scaler.transform(latest_features)

# Predict with all models
next_day_predictions = {}
print("\n🔮 NEXT-DAY PRICE PREDICTIONS")
print("="*60)

for name, model in models.items():
    pred = model.predict(latest_features_scaled)[0]
    next_day_predictions[name] = pred
    print(f"  {name}: ${pred:.2f}")

# Ensemble prediction for next day
ensemble_next_day = np.sum([next_day_predictions[name] * weights[idx] 
                             for idx, name in enumerate(predictions.keys())])

current_price = features_df['Close'].iloc[-1]
predicted_change = ((ensemble_next_day - current_price) / current_price) * 100

print("\n" + "="*60)
print(f"📊 Current Price: ${current_price:.2f}")
print(f"🎯 Ensemble Prediction: ${ensemble_next_day:.2f}")
print(f"📈 Predicted Change: {predicted_change:+.2f}%")
print("="*60)

## 12. 🎯 BUY/SELL SIGNAL GENERATION

### This is what you need for paper trading!

In [ ]:
# Calculate prediction agreement
predictions_array = np.array(list(next_day_predictions.values()))
pred_changes = ((predictions_array - current_price) / current_price) * 100

# Count bullish and bearish predictions
bullish_count = np.sum(pred_changes > 0)
bearish_count = np.sum(pred_changes < 0)
total_models = len(predictions_array)

# Calculate confidence
bullish_confidence = bullish_count / total_models
bearish_confidence = bearish_count / total_models

# Generate signal
if predicted_change > BUY_THRESHOLD and bullish_confidence >= CONFIDENCE_THRESHOLD:
    signal = "🟢 STRONG BUY"
    signal_color = 'green'
    action = "BUY"
elif predicted_change > 0 and bullish_confidence >= 0.5:
    signal = "🟢 BUY"
    signal_color = 'lightgreen'
    action = "BUY"
elif predicted_change < SELL_THRESHOLD and bearish_confidence >= CONFIDENCE_THRESHOLD:
    signal = "🔴 STRONG SELL"
    signal_color = 'red'
    action = "SELL"
elif predicted_change < 0 and bearish_confidence >= 0.5:
    signal = "🔴 SELL"
    signal_color = 'lightcoral'
    action = "SELL"
else:
    signal = "🟡 HOLD"
    signal_color = 'yellow'
    action = "HOLD"

# Calculate position size based on confidence
max_confidence = max(bullish_confidence, bearish_confidence)
position_size_pct = max_confidence * 100

print("\n" + "#"*80)
print("#" + " "*78 + "#")
print("#" + " "*20 + "🎯 TRADING RECOMMENDATION FOR PAPER TRADING 🎯" + " "*12 + "#")
print("#" + " "*78 + "#")
print("#"*80)
print("\n")

print(f"📊 Stock: {TICKER}")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💰 Current Price: ${current_price:.2f}")
print(f"🎯 Predicted Price: ${ensemble_next_day:.2f}")
print(f"📈 Expected Change: {predicted_change:+.2f}%")

print("\n" + "-"*80)
print(f"\n⚡ RECOMMENDATION: {signal}")
print(f"   Action: {action}")
print(f"   Confidence: {max_confidence*100:.1f}%")
print(f"   Position Size: {position_size_pct:.1f}% of capital")
print("\n" + "-"*80)

print("\n📊 Model Agreement:")
print(f"   Bullish models: {bullish_count}/{total_models} ({bullish_confidence*100:.1f}%)")
print(f"   Bearish models: {bearish_count}/{total_models} ({bearish_confidence*100:.1f}%)")

print("\n📝 Individual Model Predictions:")
for name, pred in next_day_predictions.items():
    change = ((pred - current_price) / current_price) * 100
    direction = "📈" if change > 0 else "📉"
    print(f"   {direction} {name}: ${pred:.2f} ({change:+.2f}%)")

print("\n" + "="*80)
print("\n💡 PAPER TRADING INSTRUCTIONS:")
print("="*80)

if action == "BUY":
    print(f"\n1. Open your paper trading account (TradingView, Webull, etc.)")
    print(f"2. Place a BUY order for {TICKER}")
    print(f"3. Suggested position: {position_size_pct:.0f}% of available capital")
    print(f"4. Entry price target: ~${current_price:.2f}")
    print(f"5. Target exit price: ${ensemble_next_day:.2f} ({predicted_change:+.2f}%)")
    print(f"6. Stop-loss: ${current_price * 0.98:.2f} (-2%)")
    print(f"7. Take-profit: ${current_price * 1.05:.2f} (+5%)")
    
elif action == "SELL":
    print(f"\n1. If you own {TICKER}, consider selling")
    print(f"2. Open your paper trading account")
    print(f"3. Place a SELL order for your {TICKER} position")
    print(f"4. Or open a SHORT position if allowed ({position_size_pct:.0f}% of capital)")
    print(f"5. Entry price: ~${current_price:.2f}")
    print(f"6. Exit target: ${ensemble_next_day:.2f}")
    
else:  # HOLD
    print(f"\n1. No strong signal detected")
    print(f"2. Keep your current positions")
    print(f"3. Wait for a clearer signal (>70% model agreement)")
    print(f"4. Monitor the stock and re-run this analysis tomorrow")

print("\n" + "="*80)
print("⚠️  RISK DISCLAIMER:")
print("   - This is for PAPER TRADING only (fake money)")
print("   - Past performance does not guarantee future results")
print("   - Stock predictions are probabilistic, not certain")
print(f"   - Model accuracy (R²): {ensemble_r2:.2%}")
print(f"   - Average prediction error (MAPE): {ensemble_mape:.2f}%")
print("="*80)
print("\n")

In [ ]:
# Visualize the trading signal
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Price prediction chart
last_30_days = features_df['Close'].iloc[-30:]
dates_30 = features_df.index[-30:]
next_date = dates_30[-1] + timedelta(days=1)

ax1.plot(dates_30, last_30_days, 'b-', linewidth=2, label='Historical Price')
ax1.plot([dates_30[-1], next_date], [current_price, ensemble_next_day], 
         'r--', linewidth=2, marker='o', markersize=10, label='Prediction')
ax1.axhline(y=current_price, color='gray', linestyle=':', alpha=0.5)
ax1.fill_between([dates_30[-1], next_date], 
                  [current_price, ensemble_next_day], 
                  alpha=0.3, color=signal_color)
ax1.set_title(f'{TICKER} - Price Prediction & Signal', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.text(next_date, ensemble_next_day, f"  ${ensemble_next_day:.2f}\n  {signal}", 
         fontsize=10, fontweight='bold', va='center')

# Model agreement pie chart
labels = ['Bullish', 'Bearish']
sizes = [bullish_count, bearish_count]
colors = ['lightgreen', 'lightcoral']
explode = (0.1 if bullish_count > bearish_count else 0, 
           0.1 if bearish_count > bullish_count else 0)

ax2.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
ax2.set_title(f'Model Agreement\n(Confidence: {max_confidence*100:.1f}%)', 
              fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 13. 📊 Final Summary

In [ ]:
print("\n" + "="*80)
print(" "*25 + "📊 ANALYSIS SUMMARY 📊")
print("="*80)

print(f"\n📈 Stock: {TICKER} - {info.get('name', 'Google')}")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Data Period: {PERIOD}")
print(f"📐 Total Data Points: {len(features_df)}")
print(f"🔧 Features Created: {len(feature_names)}")

print(f"\n💰 Price Information:")
print(f"   Current Price: ${current_price:.2f}")
print(f"   Predicted Price (Next Day): ${ensemble_next_day:.2f}")
print(f"   Expected Change: {predicted_change:+.2f}%")
print(f"   52-Week High: ${df['High'].iloc[-252:].max():.2f}")
print(f"   52-Week Low: ${df['Low'].iloc[-252:].min():.2f}")

print(f"\n🤖 Models Trained: {len(models)}")
print(f"   1. Linear Regression")
print(f"   2. Ridge Regression")
print(f"   3. Lasso Regression")
print(f"   4. Random Forest")
print(f"   5. XGBoost")
print(f"   6. Gradient Boosting")
print(f"   7. Extra Trees")
print(f"   8. AdaBoost")

print(f"\n🏆 Best Individual Model: {best_model_name}")
print(f"   R² Score: {best_r2:.4f}")
print(f"   MAPE: {metrics_df.loc[best_model_name, 'MAPE']:.2f}%")

print(f"\n🎯 Ensemble Model Performance:")
print(f"   R² Score: {ensemble_r2:.4f}")
print(f"   RMSE: ${ensemble_rmse:.2f}")
print(f"   MAE: ${ensemble_mae:.2f}")
print(f"   MAPE: {ensemble_mape:.2f}%")

print(f"\n⚡ TRADING SIGNAL: {signal}")
print(f"   Recommended Action: {action}")
print(f"   Confidence Level: {max_confidence*100:.1f}%")
print(f"   Bullish Models: {bullish_count}/{total_models}")
print(f"   Bearish Models: {bearish_count}/{total_models}")

print("\n🔝 Top 5 Important Features:")
for i, row in feature_importance_df.head(5).iterrows():
    print(f"   {i+1}. {row['Feature']}: {row['Importance']:.4f}")

print("\n" + "="*80)
print("\n✅ Analysis Complete! Use the recommendation above for paper trading.")
print("\n💡 Pro Tips:")
print("   • Always use stop-loss orders to limit losses")
print("   • Never invest more than you can afford to lose")
print("   • Paper trade for at least 3-6 months before using real money")
print("   • Diversify your portfolio across multiple stocks")
print("   • Re-run this analysis daily for updated signals")
print("="*80)
print("\n")

## 🎯 What to Do Next?

### For Paper Trading:

1. **Open a Paper Trading Account**
   - TradingView (free)
   - Webull (paper trading mode)
   - TD Ameritrade thinkorswim

2. **Follow the Recommendation**
   - Use the BUY/SELL/HOLD signal from Section 12
   - Set the suggested stop-loss and take-profit levels
   - Track your trades in a spreadsheet

3. **Run Daily**
   - Re-run this notebook every day before market open
   - Adjust your positions based on new signals
   - Track your win rate and performance

4. **After 100+ Trades (3-6 months)**
   - Calculate your total return
   - If profitable and consistent, consider real trading with small amounts
   - If not profitable, improve the model or strategy

### To Improve This Model:

1. **Add More Data Sources**
   - Sentiment analysis from news/Twitter
   - Economic indicators (interest rates, GDP)
   - Competitor stock prices

2. **Try Different Features**
   - More technical indicators
   - Market breadth indicators
   - Sector performance

3. **Advanced Techniques**
   - LSTM neural networks for sequences
   - Transformer models
   - Reinforcement learning for trading strategy

### ⚠️ Important Reminders:

- 📊 **This is NOT financial advice**
- 🎓 **Educational purposes only**
- 💰 **Use paper trading first** (at least 6 months)
- 📉 **Past performance ≠ future results**
- 🛡️ **Always use risk management** (stop-loss, position sizing)
- 📈 **Even 60% accuracy can lose money** (due to fees and slippage)

Good luck with your paper trading! 🚀